# Model Training Notebook — Keris-RD-Net (Main Model)

This notebook trains and evaluates **Keris-RD-Net (Main Model)** for the Keris image classification task.  
It has been refactored for **reproducibility** and to serve as a clean **appendix artifact** for journal submission.

## Recommended folder conventions
- **Input data**: keep dataset paths configurable (see the *Configuration* cell).
- **Outputs / artifacts**: write all run artifacts under `artifacts/02_keris_rd_net/` (created automatically below).

## Reproducibility checklist
- Fixed random seed (NumPy / framework seed)
- Best-effort deterministic operations (may vary by GPU/driver)
- Logged environment versions


In [ ]:
# --- Environment & reproducibility (TensorFlow) ---
import os, sys, platform, random
import numpy as np

SEED = int(os.environ.get("SEED", "42"))
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ.setdefault("TF_DETERMINISTIC_OPS", "1")  # best-effort determinism

random.seed(SEED)
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

# GPU memory growth (safe default)
gpus = tf.config.list_physical_devices("GPU")
for g in gpus:
    try:
        tf.config.experimental.set_memory_growth(g, True)
    except Exception:
        pass

print("Python   :", sys.version.split()[0])
print("Platform :", platform.platform())
print("NumPy    :", np.__version__)
print("TF       :", tf.__version__)
print("GPUs     :", gpus if gpus else "None (CPU)")

In [ ]:
# --- Configuration (paths & artifact directory) ---
from pathlib import Path

# Project root: by default, current working directory
PROJECT_ROOT = Path.cwd()

# Edit these paths if needed
DATA_ROOT = PROJECT_ROOT / "dataset"      # <-- set your dataset root here
NPY_ROOT  = PROJECT_ROOT / "npy"          # <-- set your .npy root here (if used)

# All outputs should go here
ARTIFACT_DIR = PROJECT_ROOT / "artifacts" / "02_keris_rd_net"
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT :", PROJECT_ROOT)
print("DATA_ROOT    :", DATA_ROOT)
print("NPY_ROOT     :", NPY_ROOT)
print("ARTIFACT_DIR :", ARTIFACT_DIR)

## Training & evaluation (original workflow)
The cells below contain the original training pipeline with minimal functional changes.


In [ ]:
import os
from glob import glob
from PIL import Image

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

# import efficientnet.keras as efn
# from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
# from tensorflow.keras.applications.resnet import ResNet152
# from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K

%matplotlib inline
import matplotlib.pyplot as plt
import random

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

In [ ]:
# Load Dataset
X_train= "x_train.npy"
X_test = "x_test.npy"
X_val = "x_valid.npy"
y_train= "y_train.npy"
y_test = "y_test.npy"
y_val = "y_valid.npy"
X_train = np.load(X_train)
X_test = np.load(X_test)
X_val = np.load(X_val)
y_train = np.load(y_train)
y_test = np.load(y_test)
y_val = np.load(y_val)
# seed_everything(1)

In [ ]:
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

In [ ]:
plt.imshow(X_train[200])

In [ ]:
# Hitung jumlah class
classes, counts = np.unique(y_train, axis=0, return_counts=True)

# Print hasilnya
print("Kelas: ", classes)
print("Jumlah: ", counts)

In [ ]:
# Hitung jumlah class
classes1, counts1 = np.unique(y_test, axis=0, return_counts=True)

# Print hasilnya
print("Kelas: ", classes1)
print("Jumlah: ", counts1)

In [ ]:
# Hitung jumlah class
classes2, counts2 = np.unique(y_val, axis=0, return_counts=True)

# Print hasilnya
print("Kelas: ", classes2)
print("Jumlah: ", counts2)

In [ ]:
counts, counts1, counts2

In [ ]:
import time
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import class_weight
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (
    SeparableConv2D, Conv2D, BatchNormalization, Activation,
    GlobalAveragePooling2D, Dense, Reshape, Multiply,
    Concatenate, Add, Dropout, Lambda
)
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
import tensorflow_addons as tfa
from tensorflow.keras.metrics import TopKCategoricalAccuracy

In [ ]:
# --- 1) Define improved dilated‐residual block ---
def inception_residual_dilated_block(x, filters, reduction=16, drop_rate=0.2, scale=0.1):
    # pre-activation
    preact = BatchNormalization()(x)
    preact = Activation('relu')(preact)

    # four lightweight branches
    branches = []
    for kernel_size, rate in [(1,1),(3,2),(3,3),(5,4)]:
        b = SeparableConv2D(filters//4, kernel_size,
                            dilation_rate=rate,
                            padding='same', use_bias=False)(preact)
        b = BatchNormalization()(b)
        b = Activation('relu')(b)
        branches.append(b)

    merged = Concatenate()(branches)
    merged = Dropout(drop_rate)(merged)
    merged = Lambda(lambda z: z * scale)(merged)

    # skip connection (projection if needed)
    skip = x
    if x.shape[-1] != filters:
        skip = Conv2D(filters, 1, padding='same', use_bias=False)(preact)
        skip = BatchNormalization()(skip)

    # add & SE‐block
    r = Add()([skip, merged])
    se = GlobalAveragePooling2D()(r)
    se = Dense(filters//reduction, activation='relu', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', use_bias=False)(se)
    se = Reshape((1,1,filters))(se)
    r = Multiply()([r, se])

    # final normalization & activation
    r = BatchNormalization()(r)
    return Activation('relu')(r)

# --- 2) Load backbone & freeze ---
base = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(512,512,3))
base.trainable = False

# --- 3) Build full model ---
inp = Input(shape=(512,512,3))
x   = base(inp, training=False)
x   = inception_residual_dilated_block(x, 768, reduction=16, drop_rate=0.2, scale=0.1)
x   = inception_residual_dilated_block(x, 768, reduction=16, drop_rate=0.2, scale=0.1)
x   = GlobalAveragePooling2D()(x)

for units in (512, 256):
    x = Dense(units, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

out = Dense(27, activation='softmax')(x)
model = Model(inputs=inp, outputs=out)

In [ ]:
# --- 4) Class weights ---
y_labels = np.argmax(y_train, axis=1)
classes  = np.unique(y_labels)
weights  = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_labels
)
class_weights = dict(zip(classes, weights))

# --- 5) Manual multiclass focal loss ---
def focal_loss(alpha=0.25, gamma=2.0):
    def loss_fn(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1-1e-7)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma)
        fl = weight * cross_entropy
        return tf.reduce_sum(fl, axis=-1)
    return loss_fn

loss_fn = focal_loss(alpha=0.25, gamma=2.0)

# --- 6) Optimizer & LR schedule ---
learning_rate_schedule = ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=100_000,
    decay_rate=0.96,
    staircase=True
)
optimizer = Adam(
    learning_rate=learning_rate_schedule,
    clipnorm=1.0, beta_1=0.9, beta_2=0.999, epsilon=1e-7
)

# --- 7) Compile with recall-focused metrics ---
model.compile(
    loss=loss_fn,
    optimizer=optimizer,
    metrics=[
        'accuracy',
        Precision(name='precision'),
        Recall(name='recall'),
        AUC(name='auc'),
        TopKCategoricalAccuracy(k=3, name='top_3_acc')
    ]
)

# --- 8) Callbacks on val_loss (mode='min' karena ingin val_loss sekecil mungkin) ---
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss', mode='min',
    patience=4, factor=0.5, min_lr=1e-6,
    cooldown=2, verbose=1
)
early_stop = EarlyStopping(
    monitor='val_loss', mode='min',
    patience=8, restore_best_weights=True,
    verbose=1
)

class TimeHistory(Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()
    def on_train_end(self, logs=None):
        print(f"Total training time: {time.time() - self.start_time:.2f} seconds")


In [ ]:
model.summary()

In [ ]:
# --- 9) Train ---
y_train = y_train.astype('float32')

history = model.fit(
    X_train, y_train,
    batch_size=8,
    epochs=100,
    validation_data=(X_val, y_val),
    class_weight=class_weights,
    callbacks=[lr_reduction, early_stop, TimeHistory()],
    verbose=1
)

In [ ]:
from sklearn.metrics import f1_score, classification_report
import numpy as np

# Predict using the model
y_pred_probs = model.predict(X_test)  # Replace X_test with your test data

# For binary classification, take the class with the highest probability (sigmoid output)
y_pred = (y_pred_probs > 0.5).astype(int)  # If sigmoid, output is a probability, threshold at 0.5

# For binary classification, y_test is already 1D, so no need for np.argmax()
y_true = y_test  # Replace y_test with the ground truth labels

# Calculate F1 Score
f1 = f1_score(y_true, y_pred, average='weighted')  # 'weighted' for handling class imbalances
print("F1 Score:", f1)

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, digits=4))

In [ ]:
import math

loss, acc, prec, rec, auc, topk = model.evaluate(X_test, y_test, verbose=1)

# Hitung F1-Score dengan benar, dan antisipasi pembagi nol
if prec + rec > 0:
    f1 = 2 * prec * rec / (prec + rec)
else:
    f1 = 0.0

print("Loss       :", loss)
print("Accuracy   :", acc)
print("Precision  :", prec)
print("Recall     :", rec)
print("AUC        :", auc)
print("Top K        :", topk)
print("F1-Score   :", f1)

In [ ]:
# model.save("4-Resnet.h5")

In [ ]:
# Hitung jumlah class
classes, counts = np.unique(y_test, axis=0, return_counts=True)

# Print hasilnya
print("Kelas: ", classes)
print("Jumlah: ", counts)

In [ ]:
import itertools
# Function to plot confusion matrix    
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# 1. Predict probabilities dari model
Y_pred_probs = model.predict(X_test)

# 2. Konversi probabilitas ke label prediksi (multiclass)
Y_pred_classes = np.argmax(Y_pred_probs, axis=1)

# 3. Siapkan label sebenarnya
# Jika y_test one-hot encoded, ubah ke indeks; 
# jika sudah 1D array, pakai langsung
if y_test.ndim > 1 and y_test.shape[1] > 1:
    Y_true = np.argmax(y_test, axis=1)
else:
    Y_true = y_test

# 4. Hitung confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)

# 5. Definisikan nama-nama class (ubah sesuai nama sebenarnya jika ada)
class_labels = [f'class {i}' for i in range(27)]

# 6. Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (27 Classes)')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# 7. Tampilkan classification report
print("Classification report for classifier %s:\n%s\n"
      % (model, classification_report(
            Y_true, 
            Y_pred_classes, 
            target_names=class_labels, 
            digits=4
        )
     )
)

In [ ]:
def get_classification_report(y_test, y_pred):
    import pandas as pd
    from sklearn.metrics import classification_report, confusion_matrix

    # 1. Hitung report dan confusion matrix
    report = classification_report(y_test, y_pred, output_dict=True)
    conf_matrix = confusion_matrix(y_test, y_pred)
    num_classes = conf_matrix.shape[0]

    # 2. Jika binary classification, unpack TN, FP, FN, TP
    if num_classes == 2:
        tn, fp, fn, tp = conf_matrix.ravel()
        print("True Negatives :", tn)
        print("False Positives:", fp)
        print("False Negatives:", fn)
        print("True Positives :", tp)
    else:
        # 3. Multiclass: print matrix dan per-class TP/FP/FN/TN
        print(f"Confusion Matrix ({num_classes} classes):")
        print(conf_matrix)
        print("\nPer-class metrics:")
        for i in range(num_classes):
            tp = conf_matrix[i, i]
            fp = conf_matrix[:, i].sum() - tp
            fn = conf_matrix[i, :].sum() - tp
            tn = conf_matrix.sum() - (tp + fp + fn)
            print(f" Class {i:2d} → TP={tp:4d}, FP={fp:4d}, FN={fn:4d}, TN={tn:4d}")

    # 4. DataFrame untuk classification report
    df_classification_report = pd.DataFrame(report).transpose()

    # 5. DataFrame untuk confusion matrix dengan label baris/kolom
    class_labels = [f"class_{i}" for i in range(num_classes)]
    df_conf_matrix = pd.DataFrame(
        conf_matrix,
        index=class_labels,
        columns=class_labels
    )

    # 6. Tampilkan matrix sebagai DataFrame
    print("\nConfusion Matrix DataFrame:")
    print(df_conf_matrix)

    return df_classification_report

In [ ]:
get_classification_report(Y_true, Y_pred_classes)

In [ ]:
import matplotlib.pyplot as plt

# 1. Ambil semua key dari history
keys = list(history.history.keys())

# 2. Cari key untuk accuracy (train) dan val_accuracy (val)
#    Sesuaikan substring kalau metric Anda namanya 'categorical_accuracy' atau lain
train_acc_key = next((k for k in keys if k == 'accuracy' or 'accuracy' in k and not k.startswith('val_')), None)
val_acc_key   = next((k for k in keys if k.startswith('val_') and 'accuracy' in k), None)

if train_acc_key is None or val_acc_key is None:
    raise ValueError(f"Metric accuracy tidak ditemukan di history.keys(): {keys}")

# 3. Plotting accuracy
plt.figure(figsize=(8,5))
plt.plot(history.history[train_acc_key], label=f"train ({train_acc_key})")
plt.plot(history.history[val_acc_key],   label=f"val   ({val_acc_key})")
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

# 4. (Opsional) Kalau mau plotting loss juga:
if 'loss' in keys and 'val_loss' in keys:
    plt.figure(figsize=(8,5))
    plt.plot(history.history['loss'],     label='train (loss)')
    plt.plot(history.history['val_loss'], label='val   (val_loss)')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

In [ ]:
keys

In [ ]:
plt.plot(history.history[keys[2]])
plt.plot(history.history[keys[8]])
plt.title('Model Precision')
plt.ylabel('precision')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history[keys[3]])
plt.plot(history.history[keys[9]])
plt.title('Model Recall')
plt.ylabel('recall')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history[keys[1]])
plt.plot(history.history[keys[7]])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history[keys[4]])
plt.plot(history.history[keys[10]])
plt.title('Model AUC')
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history[keys[5]])
plt.plot(history.history[keys[11]])
plt.title('Top K Categorical Accuracy')
plt.ylabel('top_3_acc')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import math
import matplotlib.pyplot as plt

# 1. Ambil list history
h = history.history

# 2. Ambil precision & recall untuk train dan val
train_prec = h['precision']
train_rec  = h['recall']
val_prec   = h['val_precision']
val_rec    = h['val_recall']

# 3. Hitung G-Mean: sqrt(precision * recall)
gmean_train = [math.sqrt(p * r) for p, r in zip(train_prec, train_rec)]
gmean_val   = [math.sqrt(p * r) for p, r in zip(val_prec,   val_rec)]

# 4. Plot
plt.figure(figsize=(8,5))
plt.plot(gmean_train, label='train G-Mean')
plt.plot(gmean_val,   label='val   G-Mean')
plt.title('Model G-Mean')
plt.xlabel('Epoch')
plt.ylabel('G-Mean')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(30, 5))
ax = ax.ravel()

for i, met in enumerate(['precision', 'recall', 'accuracy', 'loss', 'auc']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:
modelh5__ = 'model.h5'
model.save('model-h5/'+modelh5__)

In [ ]:
# experiment.end()

## Notes
- Keep dataset paths and output paths configurable for reproducibility.
- If you publish this notebook, ensure no private paths or secrets are embedded.
